### Quantum Clustering Algorithm
Seth Lloyd, Masoud Mohseni, Patrick Rebentrost, quant-ph/1307.0411

A variant of this algorithm was put out by Svore et al here: quant-ph/1401.2142

Ewin Tang dequantized this algorithm: cs/1811.00414. Nevertheless it remains an interesting exercise for Qumquat.

Estimates the distance of a vector $\vec u$ to the centroid of some vectors $\vec v_i$.

In [1]:
import qumquat as qq
import random, math

Generate data. We are given QRAM state preparation and query access to $\vec u$, $\vec v_i$, a vector of the norms $|\vec v_i|$. We also know $|\vec u|$.

In [2]:
N = 10 # dimension
M = 10 # number of vectors in cluster

u = [random.uniform(-5, 5) for i in range(N)]
u_norm = math.sqrt(sum([x**2 for x in u]))

vs = []
for j in range(M):
    vs.append([random.uniform(-1,1) for i in range(N)])

v_norms = [math.sqrt(sum([x**2 for x in vs[j]])) for j in range(M)]

Let's calculate distance classically. This takes time linear in $N,M$.

In [3]:
delta = [u[i] for i in range(N)]
for j in range(M):
    for i in range(N):
        delta[i] += vs[j][i]/M

D_classical = math.sqrt(sum([x**2 for x in delta]))
print("Classical Distance:", D_classical)

Classical Distance: 7.409689790954205


$\newcommand{\ket}[1]{|#1\rangle}\newcommand{\bra}[1]{\langle#1|}$
An important part of the algorithm is the state:
$$\ket{\phi} = \frac{1}{\sqrt{Z}} \left( |\vec u| \ket{0} - \frac{1}{\sqrt{M}} \sum_j |\vec v_j|\ket{j}\right)$$

Where $Z = |\vec u|^2 + \sum_i |\vec v_i|^2$ and the $\vec v_i$ are numbered 1 to $M$.

To prepare this state, we first prepare a precursor state:
$$\ket{\phi_0} = \frac{1}{\sqrt{2}} \left( \ket{0} - \frac{1}{\sqrt{M}} \sum_j \ket{j}\right)$$

In [4]:
def make_phi_zero():
    tmp = qq.reg([0,1])
    with qq.q_if(tmp): phi = qq.reg(range(1,M+1))
    tmp.clean(phi > 0)
    return phi

To prepare $\ket{\phi}$ we can use the following Hamiltonian, which acts on $\ket{\phi_0}$ and the sign bit of some temporary register `tmp`, i.e. `tmp[-1]`.

$$H = \left( |\vec u|\ket{0}\bra{0} + \frac{1}{\sqrt{M}} \sum_j |\vec v_j| \ket{j}\bra{j} \right) \otimes \sigma_X$$ 

This hamiltonian is nearly diagonal - we just need to hadamard the last bit to diagonalize it. This makes simulating the hamiltonian for time $t$ possible with a QRAM query to the vector of $|\vec v_i|$.

In [5]:
def apply_hamiltonian(t, phi, tmp):
    tmp.had(-1) # hadamard the sign bit
    with qq.q_if(phi == 0):
        qq.phase(t*u_norm * tmp)
    with qq.q_if(phi > 0):
        qq.phase(t*(phi-1).qram(v_norms) * tmp)
    tmp.had(-1) 

If we apply $H$ for time $t$ to $\ket{\phi_0}$ and `tmp[-1]` we obtain:

$$\frac{1}{\sqrt{2}}  \left(  \cos(|\vec u| t)\ket{0} - \frac{1}{\sqrt{M}} \sum_j \cos(|\vec v_j|t) \ket{j}  \right) \otimes \ket{0} - \frac{i}{\sqrt{2}}  \left(  \sin(|\vec u| t)\ket{0} - \frac{1}{\sqrt{M}} \sum_j \sin(|\vec v_j|t) \ket{j}  \right) \otimes \ket{1} $$

If $t$ is small enough, $|\vec u|t, |\vec v_j|t \ll 1$, then the small angle approximation holds.
$$ \sin(|\vec u|t) \approx |\vec u|t,\hspace{1cm} \sin(|\vec v_j|t) \approx |\vec v_j|t$$

That way if we postselect on measuring $\ket{1}$ for `tmp[-1]`, we obtain $\ket{\phi_0}$.

The probability of success of postselection is $Z^2 t^2$, letting us obtain an extremely crude estimate for $Z$ (the probability can only be estimated to additive error and $1/t^2$ will be huge).

In [6]:
t = 1e-1 * min(1/u_norm, min([1/v_norm for v_norm in v_norms]))
print("t =", t)
print("sin(|u|*t) - |u|*t is", math.sin(u_norm*t) - u_norm*t, "\n")

def make_phi():
    phi = make_phi_zero()
    tmp = qq.reg(1) # must be 1, that way we get +-1
    
    apply_hamiltonian(t, phi, tmp)
    
    prob = qq.postselect(tmp[-1] == 1)
    print("Prepared |phi> with probability", prob)
    
    Z_estimate = math.sqrt(prob)/t
    return phi, Z_estimate

t = 0.012688188022442624
sin(|u|*t) - |u|*t is -0.0001665833531718508 



We can also compute $Z$ classically in time linear in $N,M$. quant-ph/1307.0411 mentions quantum counting as a method for estimating $Z$, but I wasn't able to figure that out. quant-ph/1401.2142 may have an answer.

In [7]:
Z_classical = u_norm**2 + sum([v_norm**2 for v_norm in v_norms])/M

Our goal is to estimate the distance:
$$D = \left| \vec u - \frac{1}{M} \sum_j \vec v_j \right|$$

Using a heavy QRAM query we can prepare the state:
$$\ket{\psi} = \frac{1}{\sqrt{2}} \left( \ket{0}\ket{\vec u}  + \frac{1}{\sqrt{M}} \sum_j \ket{j}\vec{v_j}\right)$$

Indeed, Qumquat does not support querying different QRAMs in superposition. We make use of the `qq.init` function which is called by `qq.reg` to perform the initialization. That way we can conditionally initialize different states $\ket{u}, \ket{v_j}$, albeit in time $M$. This highlights that creating $\ket{\psi}$ demands either strong sparsity assumptions or a ridiculous piece of QRAM hardware.

We store $\ket{\psi}$ in two variables: `psi_key` and `psi_value`. 

In [8]:
def make_psi():

    # prepare ( |0> + M^(-1/2) sum_{j=1}^M |j> ) * 2^{-1/2}
    tmp = qq.reg([0,1])
    with qq.q_if(tmp): psi_key = qq.reg(range(1,M+1))
    tmp.clean(psi_key > 0)

    psi_value = qq.reg(0)

    with qq.q_if(psi_key == 0):
        qq.init(psi_value, {i:u[i] for i in range(N)})

    for j in range(M):
        with qq.q_if(psi_key-1 == j):
            qq.init(psi_value, {i:vs[j][i] for i in range(N)})


    return psi_key, psi_value

The magnitude squared of the inner product of `psi_key` and`phi` is:
$$ \left| \vec u - \frac{1}{M} \sum_j \vec v_j \right|^2 (|\vec u|^2 + \sum_i |\vec v_i|^2  ) = 2 D^2 Z $$

We can estimate the inner product with the swap test. The probability we estimate is $(2 D^2 Z + 1)/2$.

In [9]:
psi_key, psi_value = make_psi()

phi, Z_estimate = make_phi()

out = qq.reg([0,1])
with qq.q_if(out):
    qq.utils.swap(psi_key, phi)

out.had(0)

p_success = qq.postselect(out == 0)
p_success = 0.2

Prepared |phi> with probability 0.005237016266100272


This completes the algorithm. The estimate for $Z$ even with really generous $t$ is very bad.

In [10]:
print("Z estimate:", Z_estimate)
print("Classically found Z:", Z_classical)

D_estimate = math.sqrt(2*Z_estimate*abs(2*p_success - 1))
D_classical_Z = math.sqrt(2*Z_classical*abs(2*p_success - 1))

print("Quantum distance estimate:", D_estimate)
print("Quantum distance estimate (classical Z):", D_classical_Z)
print("Classically computed distance:", D_classical)

Z estimate: 5.703511948547743
Classically found Z: 65.26743825334496
Quantum distance estimate: 2.6161449383123427
Quantum distance estimate (classical Z): 8.849911067576553
Classically computed distance: 7.409689790954205
